## Setup: Load API key


In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file with UTF-8 encoding
load_dotenv(encoding='utf-8')

# Get the API key from environment
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

## Initialize LLM


In [ ]:
from langchain_openai import ChatOpenAI

# Create the LLM with explicit API key
llm = ChatOpenAI(model="gpt-4o", api_key=OPENAI_API_KEY)

## Define tool and bind to LLM


In [ ]:
def multiply(a: int, b: int) -> int:
    """
    Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b
llm_with_tools = llm.bind_tools([multiply])

## Build router graph


In [ ]:
from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END
from langgraph.graph import MessagesState
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition

# Node
def tool_calling_llm(state: MessagesState):
    return {
        "messages": [llm_with_tools.invoke(state["messages"])]
    }

# Build graph
builder = StateGraph(MessagesState)
builder.add_node("tool_calling_llm", tool_calling_llm)
builder.add_node("tools", ToolNode([multiply]))
builder.add_edge(START, "tool_calling_llm")
builder.add_conditional_edges(
    "tool_calling_llm",
    tools_condition,   # routes to 'tools' if the output is a tool call, or END otherwise
)
builder.add_edge("tools", END)

graph = builder.compile()

# View graph
display(Image(graph.get_graph().draw_mermaid_png()))

## Example 1: Tool call triggered


In [ ]:
from langchain_core.messages import HumanMessage

messages = [HumanMessage(content="Multiply 3 and 4.")]
messages = graph.invoke({"messages": messages})

for m in messages["messages"]:
    m.pretty_print()

================================ Human Message =================================

Multiply 3 and 4.
================================== Ai Message ==================================
Tool Calls:
  multiply (call_F3VAR2N6RemAqZGYHOsBtA76)
 Call ID: call_F3VAR2N6RemAqZGYHOsBtA76
  Args:
    a: 3
    b: 4
================================= Tool Message =================================
Name: multiply

12


## Example 2: Regular response
